# Setup

In [1]:
import re
import numexpr
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from IPython.display import display, Markdown

In [2]:
def print_text(text):
    return display(Markdown(f'<div style="font-size: 17px;">\n\n{text}\n\n</div>'))

# LLM Setup

In [3]:
# Use HuggingFaceEndpoint - automatic routing based on availability
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.7,
)

chat_llm = ChatHuggingFace(llm=llm)

# Langchain Quick Intro

## Asking a simple question

In [4]:
# Build prompt template
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

# we chain together the prompt -> LLM with LCEL (more on this later)
llm_chain = prompt | chat_llm

question = "Which NFL team won the Super Bowl in the 2010 season?"

text = (llm_chain.invoke(question))
print_text(text.content)

<div style="font-size: 17px;">

The Green Bay Packers won Super Bowl XLV (45) in the 2010 season by defeating the Pittsburgh Steelers 31-25 on February 6, 2011.

</div>

## Asking multiple questions

In [5]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

res = llm_chain.batch(qs)

In [6]:
for question, response in zip(qs, res):
    print("="*100)
    print(f"QUESTION: {question['question']}")
    print(f"RESPONSE: {response.content}")
    print("="*100 + "\n")

QUESTION: Which NFL team won the Super Bowl in the 2010 season?
RESPONSE: The Green Bay Packers won Super Bowl XLV (45) in the 2010 season by defeating the Pittsburgh Steelers with a score of 31-25.

QUESTION: If I am 6 ft 4 inches, how tall am I in centimeters?
RESPONSE: To convert your height from feet and inches to centimeters, we need to perform the following steps:

1. Convert feet to inches: 1 foot = 12 inches. 
2. Convert your height from feet and inches to inches: 6 feet = 6 * 12 = 72 inches + 4 inches = 76 inches.
3. Convert inches to centimeters: 1 inch = 2.54 centimeters. 
4. Multiply the number of inches by the conversion factor to get the height in centimeters: 76 inches * 2.54 centimeters/inch = 193.04 centimeters.

You are approximately 193.04 centimeters tall.

QUESTION: Who was the 12th person on the moon?
RESPONSE: Unfortunately, I couldn't find any information about a list of people who walked on the moon ranked by the number of the person. The Apollo missions that l

# Langchain Prompt Templates

## Simple prompt

In [7]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In [8]:
text = chat_llm.invoke(prompt)
print_text(text.content)

<div style="font-size: 17px;">

The libraries and model providers that offer Large Language Models (LLMs) are:

1. Hugging Face's `transformers` library
2. OpenAI using the `openai` library
3. Cohere using the `cohere` library

</div>

## Prompt Templates

In [9]:
template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [10]:
final_prompt = prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
print_text(final_prompt)

<div style="font-size: 17px;">

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 

</div>

In [11]:
output_parser = StrOutputParser()
chain = prompt_template | chat_llm | output_parser

response = chain.invoke({"query": "Which libraries and model providers offer LLMs?"})
print_text(response)

<div style="font-size: 17px;">

The libraries and model providers that offer Large Language Models (LLMs) are:

1. Hugging Face's `transformers` library
2. OpenAI using the `openai` library
3. Cohere using the `cohere` library

</div>

## Few-shot Training

In [12]:
prompt = PromptTemplate(
    template=(
        "Create an FAQ in Markdown based on the following questions and answers:\n"
        "Q1: What is your return policy?\n"
        "A1: We accept returns within 30 days with receipt.\n"
        "Q2: Do you ship internationally?\n"
        "A2: Yes, we ship to over 50 countries.\n"
        "Q3: How can I track my order?\n"
        "A3: Use the tracking link in your confirmation email."
    ),
    input_variables=[]  # No variables needed yet
)

chain = prompt | chat_llm | output_parser
response = chain.invoke({})  # Empty dict since no input variables
print_text(response)

<div style="font-size: 17px;">

**Frequently Asked Questions**
============================

### Returns and Refunds

#### Q: What is your return policy?
A: We accept returns within 30 days with receipt.

### Shipping

#### Q: Do you ship internationally?
A: Yes, we ship to over 50 countries.

### Order Tracking

#### Q: How can I track my order?
A: Use the tracking link in your confirmation email.

### Additional Information
-----------------------------

*   For further assistance, please contact our customer service team.
*   Please note that some countries may have additional import fees or taxes, which are the responsibility of the recipient.
*   If you have any questions or concerns about your order, feel free to reach out to us.

</div>

In [13]:
prompt_str = """
Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:

# Frequently Asked Questions

## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---

Now generate the FAQ section for this data:
Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.

"""

# Create a PromptTemplate with no input variables (static prompt)
prompt = PromptTemplate(template=prompt_str, input_variables=[])

chain = prompt | chat_llm | output_parser
response = chain.invoke({})  # Empty dict since no input variables
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions

## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select gift wrapping during **checkout**.

---

You can use the above FAQ section in your documentation or website. The anchor links (#1-what-payment-methods-do-you-accept, #2-can-i-change-my-shipping-address-after-ordering, #3-do-you-offer-gift-wrapping) can be used to link to the respective questions from other parts of your content.

</div>

## Few Shot Prompt Templates

In [14]:
# Simplified examples with just 2 variables instead of 9
examples = [
    {
        "input": (
            "Q1: What is your return policy?\n"
            "A1: We accept returns within 30 days with the original receipt."
        ),
        "output": (
            "## [1. What is your return policy?](#1-what-is-your-return-policy)\n\n"
            "We accept returns within **30 days** with the original **receipt**."
        )
    },
    {
        "input": (
            "Q2: Do you ship internationally?\n"
            "A2: Yes, we ship to over 50 countries worldwide."
        ),
        "output": (
            "## [2. Do you ship internationally?](#2-do-you-ship-internationally)\n\n"
            "Yes, we ship to over **50 countries** worldwide."
        )
    },
    {
        "input": (
            "Q3: How can I track my order?\n"
            "A3: After your order is shipped, you'll receive a tracking link via email."
        ),
        "output": (
            "## [3. How can I track my order?](#3-how-can-i-track-my-order)\n\n"
            "After your order is shipped, you'll receive a **tracking link** via email."
        )
    }
]

# Much simpler example template with only 2 variables
example_template = """Example input:
{input}

Example Output:
# Frequently Asked Questions\n\n
{output}
"""

# Create prompt template for examples
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_template
)

# Instructions prefix
prefix = """Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**
"""

# User input format and instructions
suffix = """
Now generate the FAQ section for this data:
{input}
"""

# Create the few-shot prompt template
faq_few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],
    example_separator="\n"
)

In [15]:
# See what the prompt looks like before sending to LLM
user_input = """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout."""

formatted_prompt = faq_few_shot.format(input=user_input)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

Example input:
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.

Example Output:
# Frequently Asked Questions


## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

Example input:
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order 

In [16]:
chain = faq_few_shot | chat_llm | output_parser

# Execute with LCEL chain
response = chain.invoke({"input": user_input})

# Display the formatted FAQ (response is already a clean string)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions


## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Please contact our support team as soon as possible to make any necessary changes.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during the checkout process to make your purchase a special gift.

</div>

## Dynamic inclusion/exclusion of examples in FewShotPromptTemplate using LengthBasedExampleSelector

In [17]:
# note we're adding more FAQ examples to each individual example here
# (how much you add isn't necessarily important - just that you're aligning
# examples as closely as possible to your use-case)
examples = [
    {
        "input": """Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.""",
        
        "output": """## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---"""
    },
    {
        "input": """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout.""",
        
        "output": """## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

---"""
    },
    {
        "input": """Q1: What are your store hours?
A1: We're open Monday through Friday from 9am to 9pm, and weekends from 10am to 7pm.
Q2: Do you offer price matching?
A2: Yes, we'll match any price from authorized retailers for identical products.
Q3: What is your warranty policy?
A3: All electronics come with a standard 1-year manufacturer warranty.""",
        
        "output": """## [1. What are your store hours?](#1-what-are-your-store-hours)

We're open **Monday through Friday** from **9am to 9pm**, and **weekends** from **10am to 7pm**.

## [2. Do you offer price matching?](#2-do-you-offer-price-matching)

Yes, we'll match any price from **authorized retailers** for **identical products**.

## [3. What is your warranty policy?](#3-what-is-your-warranty-policy)

All electronics come with a standard **1-year manufacturer warranty**.

---"""
    },
    {
        "input": """Q1: How do I contact customer service?
A1: You can reach us at support@example.com or call 555-123-4567.
Q2: Do you offer same-day delivery?
A2: Yes, for orders placed before 2pm in selected metro areas.
Q3: How do I cancel an order?
A3: Log into your account and cancel within 1 hour of placing the order.""",
        
        "output": """## [1. How do I contact customer service?](#1-how-do-i-contact-customer-service)

You can reach us at **support@example.com** or call **555-123-4567**.

## [2. Do you offer same-day delivery?](#2-do-you-offer-same-day-delivery)

Yes, for orders placed before **2pm** in selected **metro areas**.

## [3. How do I cancel an order?](#3-how-do-i-cancel-an-order)

Log into your account and cancel within **1 hour** of placing the order.

---"""
    },
    {
        "input": """Q1: Do you offer student discounts?
A1: Yes, students with valid ID receive 15% off all purchases.
Q2: What is your privacy policy?
A2: We never share your personal information with third parties without consent.
Q3: Are your products environmentally friendly?
A3: We use recyclable packaging and offer carbon-neutral shipping options.""",
        
        "output": """## [1. Do you offer student discounts?](#1-do-you-offer-student-discounts)

Yes, students with valid ID receive **15% off** all purchases.

## [2. What is your privacy policy?](#2-what-is-your-privacy-policy)

We **never share** your personal information with third parties without consent.

## [3. Are your products environmentally friendly?](#3-are-your-products-environmentally-friendly)

We use **recyclable packaging** and offer **carbon-neutral shipping** options.

---"""
    },
    {
        "input": """Q1: How do I apply for a refund?
A1: Submit your request through our customer portal with your order number and reason.
Q2: Do you have a loyalty program?
A2: Yes! Earn 1 point for every dollar spent and redeem for discounts.
Q3: What are the system requirements?
A3: Our software requires Windows 10/11 or macOS 10.15+, 8GB RAM, and 2GB storage.""",
        
        "output": """## [1. How do I apply for a refund?](#1-how-do-i-apply-for-a-refund)

Submit your request through our **customer portal** with your **order number** and reason.

## [2. Do you have a loyalty program?](#2-do-you-have-a-loyalty-program)

Yes! Earn **1 point** for every dollar spent and redeem for **discounts**.

## [3. What are the system requirements?](#3-what-are-the-system-requirements)

Our software requires **Windows 10/11** or **macOS 10.15+**, **8GB RAM**, and **2GB storage**.

---"""
    },
    {
        "input": """Q1: How long does shipping take?
A1: Standard shipping takes 3-5 business days, and express shipping takes 1-2 business days.
Q2: Do you have physical stores?
A2: Yes, we have 12 locations across North America. Find the nearest one on our website.
Q3: How do I reset my password?
A3: Click 'Forgot Password' on the login page and follow the email instructions.""",

        "output": """## [1. How long does shipping take?](#1-how-long-does-shipping-take)

Standard shipping takes **3-5 business days**, and express shipping takes **1-2 business days**.

## [2. Do you have physical stores?](#2-do-you-have-physical-stores)

Yes, we have **12 locations** across North America. Find the nearest one on our **website**.

## [3. How do I reset my password?](#3-how-do-i-reset-my-password)

Click **'Forgot Password'** on the login page and follow the email instructions.

---"""
    },
    {
        "input": """Q1: What's your wholesale policy?
A1: For orders over $500, contact our wholesale department for special pricing.
Q2: How can I become a vendor?
A2: Fill out the vendor application form on our Partners page for consideration.
Q3: Do you offer installation services?
A3: Yes, professional installation is available for an additional fee in most areas.""",
        
        "output": """## [1. What's your wholesale policy?](#1-whats-your-wholesale-policy)

For orders over **$500**, contact our **wholesale department** for special pricing.

## [2. How can I become a vendor?](#2-how-can-i-become-a-vendor)

Fill out the **vendor application form** on our **Partners page** for consideration.

## [3. Do you offer installation services?](#3-do-you-offer-installation-services)

Yes, **professional installation** is available for an additional fee in **most areas**.

---"""
    }
]

In [18]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=700  # this sets the max length that examples should be
)

In [19]:
some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


In [20]:
# Then create the dynamic prompt template
dynamic_faq_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],  # simplified to just one variable
    example_separator="\n"
)

In [21]:
# Using simplified input format
user_input = """Q1: What payment methods do you accept?
A1: We accept Visa, Mastercard, PayPal, and Apple Pay.
Q2: Can I change my shipping address after ordering?
A2: Only if your order hasn't shipped yet. Contact support ASAP.
Q3: Do you offer gift wrapping?
A3: Yes! You can select gift wrapping during checkout."""

formatted_prompt = dynamic_faq_prompt.format(input=user_input)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---

Example input:
Q1: What payment methods do you accept?
A1: We 

In [22]:
dynamic_chain = dynamic_faq_prompt | chat_llm | output_parser

# Execute with LCEL chain
response = dynamic_chain.invoke({"input": user_input})
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions


## [1. What payment methods do you accept?](#1-what-payment-methods-do-you-accept)

We accept **Visa**, **Mastercard**, **PayPal**, and **Apple Pay**.

## [2. Can I change my shipping address after ordering?](#2-can-i-change-my-shipping-address-after-ordering)

Only if your order hasn't shipped yet. Contact support **ASAP**.

## [3. Do you offer gift wrapping?](#3-do-you-offer-gift-wrapping)

Yes! You can select **gift wrapping** during checkout.

</div>

## LengthBasedExampleSelector working when user input is much longer and using more of the context window

In [23]:
# Using simplified input format with very long questions and answers
user_input = """Q1: What are all the different payment methods that you accept for online purchases, including credit cards, digital wallets, bank transfers, and any special financing options that might be available for customers?
A1: We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For credit cards, we accept Visa, Mastercard, American Express, and Discover. We also support digital wallet payments through PayPal, Apple Pay, Google Pay, Samsung Pay, and Amazon Pay. Additionally, we offer bank transfer options including ACH transfers, wire transfers, and direct debit for customers who prefer traditional banking methods. For larger purchases, we provide financing options through Affirm, Klarna, and our own in-house financing program with flexible payment plans ranging from 6 to 36 months. We also accept cryptocurrency payments including Bitcoin, Ethereum, and several other major cryptocurrencies for tech-savvy customers.
Q2: Is it possible for me to modify or completely change my shipping address after I have already placed and confirmed my order, and if so, what are the specific conditions, timeframes, and procedures that I need to follow?
A2: Yes, it is possible to modify your shipping address, but this depends entirely on the current status of your order in our fulfillment process. If your order has not yet been processed by our warehouse team and is still in 'pending' or 'confirmed' status, you can easily change the shipping address by logging into your account and accessing the order management section. However, once your order enters the 'processing' phase and our warehouse team begins preparing your items for shipment, address changes become much more complicated and may not be possible. If your order has already shipped, unfortunately we cannot redirect the package to a different address, but you can contact the shipping carrier directly to arrange for package interception or redirection services, though additional fees may apply. For the best chance of successful address modification, we strongly recommend contacting our customer support team immediately at support@company.com or calling our toll-free number.
Q3: Do you provide gift wrapping services for the items that I purchase, and if you do, what are the different options available, what are the costs involved, and can I include personalized messages or special requests?
A3: Absolutely! We offer comprehensive gift wrapping services to make your purchases extra special for any occasion. We have several gift wrapping options available: our standard gift wrap features elegant wrapping paper in various colors and patterns with matching ribbon and a bow for an additional $4.99 per item. Our premium gift wrap option includes luxury wrapping paper, silk ribbon, and decorative embellishments for $9.99 per item. For special occasions, we offer themed wrapping for holidays, birthdays, weddings, and baby showers at $7.99 per item. You can also add personalized gift messages up to 250 characters at no additional cost, and we'll include them on beautiful greeting cards. For an extra $2.99, you can upload custom messages or even photos to be printed on special cards. All gift-wrapped items are carefully packaged to ensure they arrive in perfect condition, and we offer discrete packaging options if you're sending gifts directly to recipients."""

In [24]:
formatted_prompt = dynamic_faq_prompt.format(input=user_input)
print(formatted_prompt)

Create a structured Markdown FAQ with anchor links, headers, and formatting conventions for readability.
Make sure to bold key terms and important information in the answers.

**Example:**

Example input:
Q1: What is your return policy?
A1: We accept returns within 30 days with the original receipt.
Q2: Do you ship internationally?
A2: Yes, we ship to over 50 countries worldwide.
Q3: How can I track my order?
A3: After your order is shipped, you'll receive a tracking link via email.

Example Output:
# Frequently Asked Questions


## [1. What is your return policy?](#1-what-is-your-return-policy)

We accept returns within **30 days** with the original **receipt**.

## [2. Do you ship internationally?](#2-do-you-ship-internationally)

Yes, we ship to over **50 countries** worldwide.

## [3. How can I track my order?](#3-how-can-i-track-my-order)

After your order is shipped, you'll receive a **tracking link** via email.

---


Now generate the FAQ section for this data:
Q1: What are all 

In [25]:
# Execute with LCEL chain
response = dynamic_chain.invoke(user_input)

# Display the formatted FAQ (response is already a clean string)
print_text(response)

<div style="font-size: 17px;">

# Frequently Asked Questions

## [1. What payment methods do you accept for online purchases?](#1-what-payment-methods-do-you-accept-for-online-purchases)

We accept a comprehensive range of payment methods to accommodate all our customers' preferences and needs. For **credit cards**, we accept **Visa**, **Mastercard**, **American Express**, and **Discover**. We also support **digital wallet payments** through **PayPal**, **Apple Pay**, **Google Pay**, **Samsung Pay**, and **Amazon Pay**. Additionally, we offer **bank transfer options** including **ACH transfers**, **wire transfers**, and **direct debit** for customers who prefer traditional banking methods. For larger purchases, we provide **financing options** through **Affirm**, **Klarna**, and our own in-house financing program with flexible payment plans ranging from **6 to 36 months**. We also accept **cryptocurrency payments** including **Bitcoin**, **Ethereum**, and several other major cryptocurrencies for tech-savvy customers.

## [2. Can I modify or change my shipping address after I have placed my order?](#2-can-i-modify-or-change-my-shipping-address-after-i-have-placed-my-order)

Yes, it is possible to modify your shipping address, but this depends entirely on the current status of your order in our fulfillment process. If your order has not yet been processed by our warehouse team and is still in **'pending' or 'confirmed' status**, you can easily change the shipping address by logging into your account and accessing the order management section. However, once your order enters the **'processing' phase** and our warehouse team begins preparing your items for shipment, address changes become much more complicated and may not be possible. If your order has already shipped, unfortunately we cannot redirect the package to a different address, but you can contact the shipping carrier directly to arrange for **package interception or redirection services**, though additional fees may apply. For the best chance of successful address modification, we strongly recommend contacting our customer support team immediately at **support@company.com** or calling our toll-free number.

## [3. Do you provide gift wrapping services for the items I purchase?](#3-do-you-provide-gift-wrapping-services-for-the-items-i-purchase)

Absolutely! We offer comprehensive **gift wrapping services** to make your purchases extra special for any occasion. We have several gift wrapping options available: our **standard gift wrap** features elegant wrapping paper in various colors and patterns

</div>

# Lanchain Chains

## Simple Chain example

In [26]:
# Create a function to handle calculations
def calculate(expression: str) -> str:
    """Calculate using numexpr, with support for basic math operations."""
    try:
        result = float(numexpr.evaluate(expression))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

# Wrap our calculation function with RunnableLambda for explicit LCEL pattern
calculate_runnable = RunnableLambda(calculate)

# Create the chain using LCEL with explicit RunnableLambda
math_chain = (
    prompt
    | chat_llm  # LLM to generate the expression
    | StrOutputParser()  # Convert to string
    | calculate_runnable  # Our calculation function wrapped in RunnableLambda
)

# Use the chain with our example
response = math_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)

The result is: 2.4116004626599237


In [27]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}")
])

basic_chain = (
    prompt
    | chat_llm  # LLM tries to calculate directly
    | StrOutputParser()
)

response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)  # The LLM tries to calculate it directly and might get it wrong!

To calculate 13 raised to the 0.3432 power, we can use the formula 13^0.3432. 

13^0.3432 = 2.10368 (rounded to 5 decimal places)


## Building Complex Chains with LCEL

In [28]:
def clean_text(text: str) -> str:
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("user", """Please paraphrase this text in the style of {style}: {text}""")
])

In [30]:
# Create the chain using LCEL
style_chain = (
    {
        "text": lambda x: clean_text(x["text"]),  # Extract and clean the text from input dict
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | prompt  # Format with our template
    | chat_llm  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

# Our input text with messy spacing
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

# Run the chain
response = style_chain.invoke({
    "text": input_text,
    "style": "a 90s rapper"
})
print(response)

Yo, listen up, it's time to spit some fire,
'Bout chains, the combo breaker, takin' it to the higher.
We're talkin' 'bout linkin' up components in a row,
Makin' one solid app, that's what the pro's know.

You got a user input, that's where it starts,
Then you add some flair, with a PromptTemplate in your arts.
Format it up right, and then you pass it on,
To an LLM, the brains of the operation, gettin' it done.

But that's not all, you can build more complex chains,
By linkin' up multiple chains, or addin' more components in the game.
You got the power, to create somethin' real,
With chains, the combo breaker, you can make it all feel.


## Using RunnableParallel and RunnablePassthrough

In [31]:
# Create two different analysis prompts
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert. Analyze the emotional tone."),
    ("user", "What's the sentiment of: {text}")
])

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarization expert."),
    ("user", "Summarize in one sentence: {text}")
])

# Use RunnableParallel to run both analyses simultaneously
analysis_chain = RunnableParallel(
    {
        "sentiment": sentiment_prompt | chat_llm | StrOutputParser(),
        "summary": summary_prompt | chat_llm | StrOutputParser(),
        "original": RunnablePassthrough()  # Pass through the original input
    }
)

# Test it
sample_text = {"text": "The product exceeded my expectations. Great quality!"}
results = analysis_chain.invoke(sample_text)

print("Sentiment:", results["sentiment"])
print("Summary:", results["summary"])
print("Original:", results["original"]["text"])

Sentiment: The sentiment of the given statement is overwhelmingly positive. 

The words used in the statement, such as "exceeded my expectations" and "Great quality", indicate a high level of satisfaction and appreciation. The tone is enthusiastic and encouraging, suggesting that the speaker is delighted with the product and would likely recommend it to others. 

The sentiment analysis score for this statement would likely be:

- Positive sentiment: 0.9 (or 90%)
- Neutral sentiment: 0.1 (or 10%)
- Negative sentiment: 0 (or 0%)
Summary: The product met a high standard, delivering excellent quality that surpassed the user's initial expectations.
Original: The product exceeded my expectations. Great quality!


## Batch Processing with LCEL

In [32]:
# Create a simple question-answering chain
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely."),
    ("user", "{question}")
])

qa_chain = qa_prompt | chat_llm | StrOutputParser()

# Batch of questions
questions = [
    {"question": "What is the capital of France?"},
    {"question": "Who wrote Romeo and Juliet?"},
    {"question": "What is the speed of light?"}
]

# Process all questions in batch
answers = qa_chain.batch(questions)

# Display results
for q, a in zip(questions, answers):
    print(f"Q: {q['question']}")
    print(f"A: {a}\n")

Q: What is the capital of France?
A: The capital of France is Paris.

Q: Who wrote Romeo and Juliet?
A: Romeo and Juliet was written by William Shakespeare.

Q: What is the speed of light?
A: The speed of light in a vacuum is approximately 299,792 kilometers per second (km/s) or 186,282 miles per second (mi/s).

